In [ ]:
# copy
import numpy as np

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

class MiniBatchLogisticRegressionNetwork:
    def __init__(self, layers, lr=0.001, epochs=1000, batch_size=5, seed=42, epsilon=0.0001):
        self.layers = layers
        self.lr = lr # współczynnik uczenia
        self.epochs = epochs
        self.batch_size = batch_size
        self.seed = seed
        self.epsilon = epsilon
        self.cost_list = []
        self.mean_cost_list = []
        self.epoch_list = []
        self.weighted_sums = list()
        self.weights = [np.random.randn(layers[i], layers[i+1]) for i in range(0, len(layers)-1) ]
        for i in self.weights:
            print(i.shape)
        self.biases = [np.zeros((1,i)) for i in layers[1:]]
        for i in self.biases:
            print(i.shape)

    def loss(self, y, y_pred):
        return -(y * np.log(y_pred) + (1 - y) * np.log(1 - y_pred))
    
    def derivative_sigmoid(self, x):
        return sigmoid(x) * (1 - sigmoid(x))
    
    def forward(self, X):
        # print('forward')
        self.weighted_sums = list()
        a = X
        for i in range(0, len(layers)-1):
            # print('i', i)
            # print('a', a.shape)
            # print('self.weights[i]', self.weights[i].shape)
            # print('dot.size',np.dot(a, self.weights[i]).shape)
            z = np.dot(a, self.weights[i]) + self.biases[i]
            print('z', z.shape)
            a = sigmoid(z)
            self.weighted_sums.append(z)
        # for i in self.weighted_sums:
        #     print(i.shape)
        return a 
    
    def backward(self, X_batch, y_batch, predicted):
        # print('predicted', predicted.shape)
        # print('y_batch', y_batch.shape)
        loss_after = y_batch - predicted #after
        #inaczej trzeba liczyć loss after - przy ostatniej warstwie jest ok ale niekoniecznie dla głębszych

        # go from last layer to first layer
        for i in range(len(layers)-2, 0, -1):

            # print('i', i)
            if( i-1 <= 0):
                weighted_sum_before_before = X_batch
            else:
                weighted_sum_before_before = self.weighted_sums[i-2]
                # weighted_sum_before = self.weighted_sums[i-1]
            
            # print('self.weighted_sums[i]', self.weighted_sums[i].shape)
            # print('self.weighted_sums[i-1]', self.weighted_sums[i-1].shape)
            # print('self.weights[i]', self.weights[i].shape)
            # print('self.weights[i-1]', self.weights[i-1].shape)
            # print('loss_after', loss_after.shape)
            loss_before = self.derivative_sigmoid(self.weighted_sums[i-1]) * (np.dot(loss_after, self.weights[i].T))
            # print('loss_before', loss_before.shape)
            delta_next = self.lr * np.dot(self.weighted_sums[i-1].T, loss_after)
            # print('delta_next', delta_next.shape)
            self.weights[i] += delta_next
            # print('self.weighted_sums[i-1]',self.weighted_sums[i-1].shape)
            # print('X_batch',X_batch.shape)
            delta_before = self.lr * np.dot(weighted_sum_before_before.T, loss_before)
            # print('delta_before', delta_before.shape)

            self.weights[i-1] += delta_before

            loss_after = loss_before


    def fit(self, X, y):
        n_samples, n_features = X.shape

        # Jeśli batch_size jest większy niż liczba próbek, to ustawiamy go na liczbę próbek -> zwyczajnie uczymy na całym zbiorze GD
        if self.batch_size > n_samples:
            self.batch_size = n_samples

        #inicjacja list na koszt i epoki
        self.cost_list = []
        self.mean_cost_list = []
        self.epoch_list = []
        cost = 0
        #ustawiamy seed
        np.random.seed(self.seed)
        #dla każdej iteracji/epoki
        for i in range(self.epochs):
            print('epochs', i)
            #tasowanie indeksów ale deterministycznie bo ustawiliśmy seed
            random_order = np.random.permutation(n_samples)
            #tasujemy X i y
            X_shuffled = X[random_order]
            y_shuffled = y[random_order]
            # lista kosztów w paczkach dla każdej epoki
            cost_list_in_batch = []

            #dla każdej paczki
            for j in range(0, n_samples, self.batch_size):

                X_batch = X_shuffled[j:j + self.batch_size]
                y_batch = y_shuffled[j:j + self.batch_size]
                # print('X_batch', X_batch.shape)
                # print('y_batch', y_batch.shape)

                predictions = self.forward(X_batch)
                self.backward(X_batch, y_batch.reshape(y_batch.shape[0], 1), predictions)
                # print ('predictions', predictions.shape)
                
                cost = np.mean(self.loss(y_batch, predictions))
                cost_list_in_batch.append(cost)

            #jeśli koszt jest mniejszy niż epsilon to stop
            if(i > 0 and abs(cost - self.cost_list[-1]) < self.epsilon ):
                break
            self.cost_list.append(cost)
            iteration_cost = np.mean(cost_list_in_batch)
            self.mean_cost_list.append(iteration_cost)
            self.epoch_list.append(i)
        print('cost', cost, 'cost_list[-1]', self.cost_list[-1], 'i', i, 'abs(cost - cost_list[-1])', abs(cost - self.cost_list[-1]))
        
    def predict(self, X):
        linear_pred = self.forward(X)
        class_pred = [0 if y <= 0.5 else 1 for y in linear_pred]
        return class_pred
        

layers = [22, 10, 5, 1]
clf = MiniBatchLogisticRegressionNetwork(layers, lr=0.001, epochs=1, batch_size=7, seed=27, epsilon=0.0001)
clf.fit(X_train,y_train)

In [2]:
import numpy as np


array1 = np.arange(70).reshape(7, 10)  # Shape (7, 10)
array2 = np.array([np.arange(10)])    # Shape (1, 10)

print(array1)
print(array2)

# Perform the addition
result = array1 + array2
result

[[ 0  1  2  3  4  5  6  7  8  9]
 [10 11 12 13 14 15 16 17 18 19]
 [20 21 22 23 24 25 26 27 28 29]
 [30 31 32 33 34 35 36 37 38 39]
 [40 41 42 43 44 45 46 47 48 49]
 [50 51 52 53 54 55 56 57 58 59]
 [60 61 62 63 64 65 66 67 68 69]]
[[0 1 2 3 4 5 6 7 8 9]]


array([[ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18],
       [10, 12, 14, 16, 18, 20, 22, 24, 26, 28],
       [20, 22, 24, 26, 28, 30, 32, 34, 36, 38],
       [30, 32, 34, 36, 38, 40, 42, 44, 46, 48],
       [40, 42, 44, 46, 48, 50, 52, 54, 56, 58],
       [50, 52, 54, 56, 58, 60, 62, 64, 66, 68],
       [60, 62, 64, 66, 68, 70, 72, 74, 76, 78]])

In [1]:
for i in range(1, 2):
    print(i)

1


In [ ]:
import numpy as np

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

class MiniBatchLogisticRegressionNetwork:
    def __init__(self, layers, lr=0.001, epochs=1000, batch_size=5, seed=42, epsilon=0.0001):
        self.layers = layers
        self.lr = lr # współczynnik uczenia
        self.epochs = epochs
        self.batch_size = batch_size
        self.seed = seed
        self.epsilon = epsilon
        self.cost_list = []
        self.mean_cost_list = []
        self.epoch_list = []
        self.weighted_sums = list()
        self.weights = [np.random.randn(layers[i], layers[i+1]) for i in range(0, len(layers)-1) ]
        for i in self.weights:
            print(i.shape)
        self.biases = [np.zeros((1,i)) for i in layers[1:]]
        for i in self.biases:
            print(i.shape)

    def loss(self, y, y_pred):
        return -(y * np.log(y_pred) + (1 - y) * np.log(1 - y_pred))
    
    def derivative_sigmoid(self, x):
        return sigmoid(x) * (1 - sigmoid(x))
    
    def forward(self, X):
        # print('forward')
        self.weighted_sums = list()
        a = X
        for i in range(0, len(layers)-1):
            # print('i', i)
            print('a', a.shape)
            print('self.weights[i]', self.weights[i].shape)
            print('self.biases[i]', self.biases[i].shape)
            print('dot.size',np.dot(a, self.weights[i]).shape)
            z = np.dot(a, self.weights[i]) + self.biases[i]
            print('z', z.shape)
            a = sigmoid(z)
            self.weighted_sums.append(z)
        # for i in self.weighted_sums:
        #     print(i.shape)
        return a 
    
    def backward(self, X_batch, y_batch, predicted):
        # print('predicted', predicted.shape)
        # print('y_batch', y_batch.shape)
        loss_after = y_batch - predicted #after
        #inaczej trzeba liczyć loss after - przy ostatniej warstwie jest ok ale niekoniecznie dla głębszych
        loss_before_history = list()
        # go from last layer to first layer
        for i in range(len(layers)-2, -1, -1):

            print('i', i)
            if( i <= 0):
                a_before = X_batch
            else:
                a_before = self.weighted_sums[i-1]
                # weighted_sum_before = self.weighted_sums[i-1]
            # print('self.weighted_sums[i]', self.weighted_sums[i].shape)
            # print('self.weighted_sums[i-1]', self.weighted_sums[i-1].shape)
            # print('a_before', a_before.shape)

            # print('self.weights[i]', self.weights[i].shape)
            # print('self.weights[i-1]', self.weights[i-1].shape)
            # print('loss_after', loss_after.shape)
            if(i == len(layers)-2):
                loss_before = np.dot(loss_after, self.weights[i].T)
            else:
                loss_before = self.derivative_sigmoid(a_before) * (np.dot(loss_after, self.weights[i].T))
            loss_before_history.append(loss_before)
                

            # delta_next = self.lr * np.dot(a_before.T, loss_after)
            # print('delta_next', delta_next.shape)
            # print('self.weights[i]', self.weights[i])
            # self.weights[i] += delta_next
            # print('self.weights[i]', self.weights[i])
            # print('self.weighted_sums[i-1]',self.weighted_sums[i-1].shape)
            # print('X_batch',X_batch.shape)

            # Tutaj mam wrażenie, że raz zmieniam wagi a potem zmieniam je jeszcze raz 


            # delta_before = self.lr * np.dot(weighted_sum_before_before.T, loss_before)
            # print('self.weights[i-1]', self.weights[i-1])
            # self.weights[i-1] += delta_before
            # print('self.weights[i-1]', self.weights[i-1])


            #dodaj biasy.

            loss_after = loss_before
        index = 0
        for i in reversed(loss_before_history):
            delta_next = self.lr * np.dot(i.T, self.weighted_sums[index])
            self.weights[index] += delta_next
            
            delta_next_b = self.lr * np.sum(i, axis=1, keepdims=True)
            print('self.biases[index]', self.biases[index].shape)
            print('delta_next_b', delta_next_b.shape)
            self.biases[index] += delta_next_b
            index += 1


    def fit(self, X, y):
        n_samples, n_features = X.shape

        # Jeśli batch_size jest większy niż liczba próbek, to ustawiamy go na liczbę próbek -> zwyczajnie uczymy na całym zbiorze GD
        if self.batch_size > n_samples:
            self.batch_size = n_samples

        #inicjacja list na koszt i epoki
        self.cost_list = []
        self.mean_cost_list = []
        self.epoch_list = []
        cost = 0
        #ustawiamy seed
        np.random.seed(self.seed)
        #dla każdej iteracji/epoki
        for i in range(self.epochs):
            print('epochs', i)
            #tasowanie indeksów ale deterministycznie bo ustawiliśmy seed
            random_order = np.random.permutation(n_samples)
            #tasujemy X i y
            X_shuffled = X[random_order]
            y_shuffled = y[random_order]
            # lista kosztów w paczkach dla każdej epoki
            cost_list_in_batch = []

            #dla każdej paczki
            for j in range(0, n_samples, self.batch_size):

                X_batch = X_shuffled[j:j + self.batch_size]
                y_batch = y_shuffled[j:j + self.batch_size]
                # print('X_batch', X_batch.shape)
                # print('y_batch', y_batch.shape)

                predictions = self.forward(X_batch)
                self.backward(X_batch, y_batch.reshape(y_batch.shape[0], 1), predictions)
                # print ('predictions', predictions.shape)
                
                cost = np.mean(self.loss(y_batch, predictions))
                cost_list_in_batch.append(cost)

            #jeśli koszt jest mniejszy niż epsilon to stop
            if(i > 0 and abs(cost - self.cost_list[-1]) < self.epsilon ):
                break
            self.cost_list.append(cost)
            iteration_cost = np.mean(cost_list_in_batch)
            self.mean_cost_list.append(iteration_cost)
            self.epoch_list.append(i)
        print('cost', cost, 'cost_list[-1]', self.cost_list[-1], 'i', i, 'abs(cost - cost_list[-1])', abs(cost - self.cost_list[-1]))
        
    def predict(self, X):
        linear_pred = self.forward(X)
        class_pred = [0 if y <= 0.5 else 1 for y in linear_pred]
        return class_pred
        

layers = [22, 10,5, 1]
clf = MiniBatchLogisticRegressionNetwork(layers, lr=0.001, epochs=1, batch_size=7, seed=27, epsilon=0.0001)
clf.fit(X_train,y_train)

In [4]:
a  = 3.134
a*2

6.268